In [2]:
!pip install schemdraw pandas openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 3.4 MB/s eta 0:00:00


### Using Schemdraw:

In [ ]:
import pandas as pd
import schemdraw
import schemdraw.elements as elm
from schemdraw.segments import *
from IPython.display import Image
import os



excel_path = os.path.join(os.getcwd(), 'Assignment_Branch_Data.xlsx')
line_data = pd.read_excel(excel_path, sheet_name='Line Data')
generator_data = pd.read_excel(excel_path, sheet_name='Generator Data')
print("Data loaded successfully!")


class CustomGenerator(elm.Element):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.segments.append(SegmentCircle((0, 0), radius=0.5))
        self.anchors['left'] = (-0.5, 0)
        self.anchors['right'] = (0.5, 0)

class CustomLoad(elm.Element):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.segments.append(Segment([(0, 0), (1.0, 0)]))
        arrowhead = [
            (1.0, 0.2),
            (1.5, 0),
            (1.0, -0.2)
        ]
        self.segments.append(SegmentPoly(arrowhead, fill=True))
        self.anchors['left'] = (0, 0)
        self.anchors['right'] = (1.5, 0)

class CustomTransformer(elm.Element):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.segments.append(SegmentCircle((-0.3, 0), radius=0.3))
        self.segments.append(SegmentCircle((0.3, 0), radius=0.3))
        self.anchors['left'] = (-0.6, 0)
        self.anchors['right'] = (0.6, 0)

buses = {}
generators = []
lines = []
loads = set()
voltage_colors = {
    345.0: "red",
    138.0: "blue",
    69.0: "green",
    34.5: "purple",
}

connections = {}
for _, row in line_data.iterrows():
    from_bus = str(row['From Bus Number'])
    to_bus = str(row['To Bus Number'])
    voltage_from = row['From Bus kV']
    voltage_to = row['To Bus kV']

    if from_bus not in buses:
        buses[from_bus] = {"voltage": voltage_from}
    else:
        if buses[from_bus]['voltage'] != voltage_from:
            print(f"Warning: Voltage mismatch for bus {from_bus} in line data. Using voltage {buses[from_bus]['voltage']} kV.")
    if to_bus not in buses:
        buses[to_bus] = {"voltage": voltage_to}
    else:
        if buses[to_bus]['voltage'] != voltage_to:
            print(f"Warning: Voltage mismatch for bus {to_bus} in line data. Using voltage {buses[to_bus]['voltage']} kV.")

    if from_bus not in connections:
        connections[from_bus] = set()
    if to_bus not in connections:
        connections[to_bus] = set()
    connections[from_bus].add(to_bus)
    connections[to_bus].add(from_bus)

    if voltage_from != voltage_to:
        lines.append({
            "from_bus": from_bus,
            "to_bus": to_bus,
            "is_transformer": True,
            "voltage1": voltage_from,
            "voltage2": voltage_to,
            "rating": row['Normal Branch Rating (MW)']
        })
    else:
        lines.append({
            "from_bus": from_bus,
            "to_bus": to_bus,
            "is_transformer": False,
            "voltage": voltage_from,
            "rating": row['Normal Branch Rating (MW)']
        })

for _, row in generator_data.iterrows():
    bus_id = str(row['Bus Number'])
    gen_voltage = row['Bus Nominal Voltage']
    generators.append({
        "id": f"{row['Bus Name']}",
        "bus": bus_id,
        "pmax": row['PMax']
    })
    if bus_id not in buses:
        buses[bus_id] = {"voltage": gen_voltage}
    else:
        if buses[bus_id]['voltage'] != gen_voltage:
            print(f"Warning: Voltage mismatch for bus {bus_id} in generator data.")

generator_buses = set(gen['bus'] for gen in generators)
for bus_id in buses:
    connected_elements = connections.get(bus_id, set())
    if len(connected_elements) == 0 and bus_id not in generator_buses:
        loads.add(bus_id)
    elif len(connected_elements) == 1 and bus_id not in generator_buses:
        loads.add(bus_id)

bus_connections = {bus_id: len(connections.get(bus_id, [])) for bus_id in buses.keys()}

d = schemdraw.Drawing()
bus_positions = {}
bus_length = 3
voltage_levels = {}

for bus_id, bus_info in buses.items():
    voltage = bus_info["voltage"]
    if voltage not in voltage_levels:
        voltage_levels[voltage] = []
    voltage_levels[voltage].append(bus_id)

grid_spacing_x = 20
grid_spacing_y = 10
voltage_level_order = sorted(voltage_levels.keys(), reverse=True)

for voltage in voltage_level_order:
    voltage_levels[voltage].sort(key=lambda bus_id: len(connections.get(bus_id, [])), reverse=True)

for v_idx, voltage in enumerate(voltage_level_order):
    bus_list = voltage_levels[voltage]
    bus_color = voltage_colors.get(voltage, "gray")
    for b_idx, bus_id in enumerate(bus_list):
        x_pos = v_idx * grid_spacing_x
        y_pos = -b_idx * grid_spacing_y
        bus_line = d.add(elm.Line().at((x_pos, y_pos)).right(bus_length).color(bus_color).linewidth(2))
        d.add(elm.Label().at((x_pos + bus_length/2, y_pos + 0.5))
              .label(f"Bus {bus_id}\n{voltage}kV", fontsize=10, halign='center', valign='center'))
        bus_positions[bus_id] = (x_pos + bus_length/2, y_pos)

def draw_right_angle_connection(start, end):
    start_x, start_y = start
    end_x, end_y = end
    if start_x == end_x or start_y == end_y:
        d.add(elm.Line().at((start_x, start_y)).to((end_x, end_y)))
    else:
        if abs(start_x - end_x) > abs(start_y - end_y):
            mid_x = end_x
            mid_y = start_y
        else:
            mid_x = start_x
            mid_y = end_y
        d.add(elm.Line().at((start_x, start_y)).to((mid_x, mid_y)))
        d.add(elm.Line().at((mid_x, mid_y)).to((end_x, end_y)))

for gen in generators:
    bus_id = gen["bus"]
    if bus_id in bus_positions:
        bus_x, bus_y = bus_positions[bus_id]
        gen_x = bus_x - bus_length - 4
        gen_y = bus_y
        gen_symbol = d.add(CustomGenerator().at((gen_x, gen_y)))
        draw_right_angle_connection(gen_symbol.anchors['right'], (bus_x - bus_length/2, bus_y))
        d.add(elm.Label().at((gen_x, gen_y - 1))
              .label(f"{gen['id']}\n{gen['pmax']}MW", fontsize=9, halign='center', valign='center'))
    else:
        print(f"Warning: Generator connected to unknown bus '{bus_id}'.")

for load_bus in loads:
    if load_bus in bus_positions:
        bus_x, bus_y = bus_positions[load_bus]
        load_x = bus_x + bus_length/2 + 2
        load_y = bus_y
        load_symbol = d.add(CustomLoad().at((load_x, load_y)))
        draw_right_angle_connection((bus_x + bus_length/2, bus_y), load_symbol.anchors['left'])
        d.add(elm.Label().at((load_x + 1.0, load_y - 1)).label("Load", fontsize=9, halign='center', valign='center'))
    else:
        print(f"Warning: Load connected to unknown bus '{load_bus}'.")

def draw_connection(start, end, is_transformer=False):
    start_x, start_y = start
    end_x, end_y = end
    if is_transformer:
        mid_x = (start_x + end_x) / 2
        mid_y = (start_y + end_y) / 2
        transformer = d.add(CustomTransformer().at((mid_x, mid_y)))
        draw_right_angle_connection((start_x, start_y), transformer.anchors['left'])
        draw_right_angle_connection(transformer.anchors['right'], (end_x, end_y))
    else:
        draw_right_angle_connection((start_x, start_y), (end_x, end_y))

for line in lines:
    if line["from_bus"] in bus_positions and line["to_bus"] in bus_positions:
        from_pos = bus_positions[line["from_bus"]]
        to_pos = bus_positions[line["to_bus"]]
        if line.get("is_transformer"):
            draw_connection(from_pos, to_pos, is_transformer=True)
        else:
            draw_connection(from_pos, to_pos)
    else:
        print(f"Warning: Connection involves unknown buses '{line['from_bus']}' and/or '{line['to_bus']}'.")

legend_start_x = max(pos[0] for pos in bus_positions.values()) + grid_spacing_x
legend_start_y = 0

d.add(elm.Label().at((legend_start_x, legend_start_y + 2)).label("Legend", fontsize=12, halign='center'))

for i, (voltage, color) in enumerate(voltage_colors.items()):
    y_pos = legend_start_y - i * 3
    d.add(elm.Line(linewidth=2).at((legend_start_x - 2, y_pos)).right(2).color(color))
    d.add(elm.Label().at((legend_start_x + 2, y_pos)).label(f"{voltage}kV Bus", fontsize=10, halign='left'))

symbol_start_y = legend_start_y - (len(voltage_colors) + 1) * 3

gen_symbol = d.add(CustomGenerator().at((legend_start_x - 1, symbol_start_y)))
d.add(elm.Label().at((legend_start_x + 2, symbol_start_y)).label("Generator", fontsize=10, halign='left'))

load_symbol = d.add(CustomLoad().at((legend_start_x - 1, symbol_start_y - 3)))
d.add(elm.Label().at((legend_start_x + 2, symbol_start_y - 3)).label("Load", fontsize=10, halign='left'))

tx_symbol = d.add(CustomTransformer().at((legend_start_x - 1, symbol_start_y - 6)))
d.add(elm.Label().at((legend_start_x + 2, symbol_start_y - 6)).label("Transformer", fontsize=10, halign='left'))

d.save('power_system_sld.png')
print("Single Line Diagram generated successfully!")

Image('power_system_sld.png')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: Excel file not found at /content/Assignment_Branch_Data.xlsx